In [43]:
# import main libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import h2o
from h2o.automl import H2OAutoML
from copulas.multivariate import VineCopula

# Step 1: Prepare the data and import it from local drive
data_train=pd.read_csv("/Users/hamed/Desktop/train.csv")
data_test=pd.read_csv("/Users/hamed/Desktop/test.csv")
data_train=data_train.drop("Name",axis=1)
data_test=data_test.drop("Name",axis=1)

train_df=pd.DataFrame(data_train)
test_df=pd.DataFrame(data_test)
try:
    train_clist=list(train_df.columns)
    train_clist.remove('Cabin')
    
    test_clist=list(test_df.columns)
    test_clist.remove('Cabin')
except:
    pass

In [44]:
# Encode data using Sklearn Label Encoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for i in train_clist:
     train_df[i]=label_encoder.fit_transform(train_df[i])
for i in test_clist:
     test_df[i]=label_encoder.fit_transform((test_df[i]))
        
        

In [45]:
try:
    new = train_df["Cabin"].str.split("/",expand = True)
    train_df["Deck"]= new[0]
    train_df["Num"]= new[1]
    train_df["side"]= new[2]
    train_df.drop(columns =["Cabin"], inplace = True)
    
    new = test_df["Cabin"].str.split("/",expand = True)
    test_df["Deck"]= new[0]
    test_df["Num"]= new[1]
    test_df["side"]= new[2]
    test_df.drop(columns =["Cabin"], inplace = True)
    
except:
    pass


In [46]:
# One last Encoding
try:
    train_df["Deck"]=label_encoder.fit_transform(train_df["Deck"])
    train_df["side"]=label_encoder.fit_transform(train_df["side"])
    train_df=train_df.drop(["PassengerId"],axis=1)

    test_df["Deck"]=label_encoder.fit_transform(test_df["Deck"])
    test_df["side"]=label_encoder.fit_transform(test_df["side"])
    test_df=test_df.drop(["PassengerId"],axis=1)
except:
    pass


In [47]:
train_df.describe()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,side
count,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000
mean,0.719199,0.399287,1.515242,29.881629,0.069596,159.551938,195.673991,133.074543,163.907742,159.578742,0.503624,4.389969,0.539054
std,0.863402,0.538345,0.840148,16.075578,0.333871,331.012706,399.189516,286.903560,341.460713,336.788574,0.500016,1.842573,0.542487
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,0.000000,0.000000,2.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,1.000000
75%,1.000000,1.000000,2.000000,39.000000,0.000000,78.000000,114.000000,45.000000,89.000000,71.000000,1.000000,6.000000,1.000000
max,3.000000,2.000000,3.000000,80.000000,2.000000,1273.000000,1507.000000,1115.000000,1327.000000,1306.000000,1.000000,8.000000,2.000000


In [48]:
test_df.describe()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,side
count,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000
mean,0.727847,0.404489,1.537526,29.728314,0.060790,116.126023,132.936404,93.252046,116.242927,102.782090,4.438859,0.536123
std,0.867673,0.533322,0.827671,15.794431,0.317186,229.723233,255.409399,191.507367,229.417571,211.462541,1.786765,0.543623
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000
50%,0.000000,0.000000,2.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,1.000000
75%,1.000000,1.000000,2.000000,38.000000,0.000000,79.000000,115.000000,49.000000,79.000000,52.000000,6.000000,1.000000
max,3.000000,2.000000,3.000000,79.000000,2.000000,842.000000,902.000000,715.000000,833.000000,796.000000,8.000000,2.000000


In [49]:
# Create a list of columns to be trained and target column
train_columns=list(train_df.columns)
train_columns.remove("Transported")
data=train_df[train_columns]
target=train_df["Transported"]

In [50]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.4, random_state=42)

In [133]:
# Scale the data to 0-1 interval
try:
    scaler = MinMaxScaler()
    X_train_scaled =scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # This should be done to test_df

    test_df=scaler.fit_transform(test_df)
except:
    pass

y_train_list = (y_train.values)

In [63]:
# Initiating the h2o 
h2o.init()

train_df_h2o=np.column_stack((X_train_scaled,y_train_list))

train_data_h2o = h2o.H2OFrame(train_df_h2o)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,25 mins 55 secs
H2O_cluster_timezone:,Europe/Rome
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,22 days
H2O_cluster_name:,H2O_from_python_hamed_e7yf4v
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.972 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [136]:
train_data_h2o["C14"]=train_data_h2o["C14"].asfactor()

In [137]:
# X are trasnfored with MinMax Scaling and our target y already satisfies the binary definition
x=list(train_data_h2o.columns)
x.remove("C14")
y="C14"



aml = H2OAutoML(max_models=20, seed=40)
aml.train(x=x,y=y, training_frame=train_data_h2o)

# this is sample code to explore CV
# # This is where list of cv preds are stored (one element per fold):
# prostate_kmeans.cross_validation_predictions()

# # However you most likely want a single-column frame including all cv preds
# prostate_kmeans.cross_validation_holdout_predictions()

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

# Get leaderboard with all possible columns
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

AutoML progress: |
18:18:29.825: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_1_AutoML_4_20230520_181829,0.895635,0.401306,0.908656,0.188206,0.359781,0.129442,2245,0.039634,StackedEnsemble
StackedEnsemble_BestOfFamily_1_AutoML_4_20230520_181829,0.893694,0.405045,0.906625,0.19458,0.361313,0.130547,2299,0.035094,StackedEnsemble
GBM_2_AutoML_4_20230520_181829,0.893662,0.405657,0.908167,0.201151,0.361684,0.130816,323,0.018817,GBM
GBM_grid_1_AutoML_4_20230520_181829_model_1,0.893334,0.407428,0.906892,0.191654,0.361869,0.130949,239,0.017209,GBM
GBM_5_AutoML_4_20230520_181829,0.892671,0.406929,0.907247,0.200337,0.362449,0.13137,248,0.011704,GBM
GBM_grid_1_AutoML_4_20230520_181829_model_3,0.891963,0.408711,0.906633,0.198636,0.363135,0.131867,305,0.014775,GBM
GBM_3_AutoML_4_20230520_181829,0.890301,0.411573,0.904562,0.200167,0.364691,0.132999,303,0.011394,GBM
GBM_4_AutoML_4_20230520_181829,0.889877,0.412678,0.903606,0.196097,0.365347,0.133478,465,0.012815,GBM
GBM_grid_1_AutoML_4_20230520_181829_model_5,0.888767,0.416089,0.902225,0.199327,0.366713,0.134478,388,0.019313,GBM
GBM_grid_1_AutoML_4_20230520_181829_model_2,0.886989,0.419035,0.901663,0.209951,0.369315,0.136393,352,0.012276,GBM


In [138]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_1_AutoML_4_20230520_181829,0.895635,0.401306,0.908656,0.188206,0.359781,0.129442,2245,0.039634,StackedEnsemble
StackedEnsemble_BestOfFamily_1_AutoML_4_20230520_181829,0.893694,0.405045,0.906625,0.19458,0.361313,0.130547,2299,0.035094,StackedEnsemble
GBM_2_AutoML_4_20230520_181829,0.893662,0.405657,0.908167,0.201151,0.361684,0.130816,323,0.018817,GBM
GBM_grid_1_AutoML_4_20230520_181829_model_1,0.893334,0.407428,0.906892,0.191654,0.361869,0.130949,239,0.017209,GBM
GBM_5_AutoML_4_20230520_181829,0.892671,0.406929,0.907247,0.200337,0.362449,0.13137,248,0.011704,GBM
GBM_grid_1_AutoML_4_20230520_181829_model_3,0.891963,0.408711,0.906633,0.198636,0.363135,0.131867,305,0.014775,GBM
GBM_3_AutoML_4_20230520_181829,0.890301,0.411573,0.904562,0.200167,0.364691,0.132999,303,0.011394,GBM
GBM_4_AutoML_4_20230520_181829,0.889877,0.412678,0.903606,0.196097,0.365347,0.133478,465,0.012815,GBM
GBM_grid_1_AutoML_4_20230520_181829_model_5,0.888767,0.416089,0.902225,0.199327,0.366713,0.134478,388,0.019313,GBM
GBM_grid_1_AutoML_4_20230520_181829_model_2,0.886989,0.419035,0.901663,0.209951,0.369315,0.136393,352,0.012276,GBM


In [139]:
m=aml.get_best_model()
m

key,value
Stacking strategy,cross_validation
Number of base models (used / total),10/20
# GBM base models (used / total),7/10
# DRF base models (used / total),2/2
# DeepLearning base models (used / total),1/7
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [146]:
test_df_h2o=np.column_stack((X_test_scaled,y_test))
test_data_h2o = h2o.H2OFrame(test_df_h2o)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.908647,0.0913529
1,0.289694,0.710306
1,0.260119,0.739881
0,0.685055,0.314945
1,0.030987,0.969013
1,0.38842,0.61158
0,0.887775,0.112225
0,0.824416,0.175584
1,0.522662,0.477338
0,0.801143,0.198857


In [157]:
# Create a new column with alternating 0s and 1s
new_column = h2o.H2OFrame([1] *4277, column_names=['13'])

# Concatenate the new column to the existing H2O frame
test_df = test_df.cbind(new_column)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [169]:
current_names = test_df.columns

# Generate the new column names
new_names = ["C" + str(i) for i in range(1, len(current_names) + 1)]

# Set the new column names
test_df.set_names(new_names)

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
0,0.5,0.666667,0.341772,0,0,0,0,0,0,0.75,0.0015873,0.5,1
0,0,0.666667,0.240506,0,0,0.00997783,0,0.869148,0,0.625,0.0021164,0.5,1
0.333333,0.5,0,0.392405,0,0,0,0,0,0,0.25,0,0.5,1
0.333333,0,0.666667,0.481013,0,0,0.93459,0,0.186074,0.451005,0.25,0.000529101,0.5,1
0,0,0.666667,0.253165,0,0.0118765,0,0.511888,0,0,0.625,0.0026455,0.5,1
0,0,0.666667,0.392405,0,0,0.676275,0.275524,0.128451,0.0741206,0.625,0.0037037,0,1
0.333333,0.5,0,0.265823,0,0,1,0,0,0,0.125,0.0010582,0,1
0.333333,0.5,0.666667,0.253165,0,0,0,0,0,0,0.375,0,0.5,1
0.333333,0.5,0,0.291139,0,0,0,0,0,0,0.375,0,0.5,1
0,0,0,0.303797,0,0,0.411308,0,0,0,0.625,0.0037037,0.5,1


In [234]:
predic=aml.predict(test_df)


h2o.save_model(aml.leader, path = "/Users/hamed/Desktop/")



stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


'/Users/hamed/Desktop/StackedEnsemble_AllModels_1_AutoML_4_20230520_181829'

In [235]:

perf = aml.leader.model_performance(test_df)
perf



ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.36748051847857355
RMSE: 0.6062017143480984
LogLoss: 1.0909391889677298
AUC: 1.0
AUCPR: 1.0
Gini: 1.0
Null degrees of freedom: 4276
Residual degrees of freedom: 4266
Null deviance: 5907.884026186273
Residual deviance: 9331.89382242996
AIC: 9353.89382242996

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.005624805770651179
       0    1     Error    Rate
-----  ---  ----  -------  ------------
0      0    0     nan      (0.0/0.0)
1      0    4277  0        (0.0/4277.0)
Total  0    4277  0        (0.0/4277.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.00562481   1         399
max f2                       0.00562481   1         399
max f0point5                 0.00562481   1         399
max accuracy                 0.00562481   1         399
max precision                0.993313     1         0
max recall                   0.00562481   1         399
max specificity              nan          nan       -1
max absolute_mcc             0.993313     0         0
max min_per_class_accuracy   nan          nan       -1
max mean_per_class_accuracy  nan          nan       -1
max tns                      0.993313     0         0
max fns                      0.993313     4231      0
max fps                      0.993313     0         0
max tps                      0.00562481   4277      399
max tnr                      nan          nan       -1
max fnr                      0.993313     0.989245  0
max fpr                      nan          nan       -1
max tpr                      0.00562481   1         399

Gains/Lift Table: Avg response rate: 100.00 %, avg score: 50.44 %
group    cumulative_data_fraction    lower_threshold    lift    cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain    cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  ------  -----------------  --------------------
1        0.0100538                   0.993058           1       1                  1                0.993377   1                           0.993377            0.0100538       0.0100538                  0       0                  0.0100538
2        0.0201076                   0.992117           1       1                  1                0.992672   1                           0.993025            0.0100538       0.0201076                  0       0                  0.0201076
3        0.0301613                   0.990057           1       1                  1                0.990874   1                           0.992308            0.0100538       0.0301613                  0       0                  0.0301613
4        0.0402151                   0.988664           1       1                  1                0.989338   1                           0.991565            0.0100538       0.0402151                  0       0                  0.0402151
5        0.0500351                   0.986067           1       1                  1                0.987454   1                           0.990758            0.00981997      0.0500351                  0       0                  0.0500351
6        0.10007                     0.974306           1       1                  1                0.981303   1                           0.986031            0.0500351       0.10007                    0       0                  0.10007
7        0.150105                    0.95736            1       1                  1                0.965882   1                           0.979315            0.0500351       0.150105                   0       0                  0.1501

In [263]:
x=predic.as_data_frame()
x=np.array(x)
fsinally=pd.DataFrame(x)
outfinal=pd.DataFrame(data_test["PassengerId"])
outfinal["Transported"]=fsinally[0]
outfinal["Transported"]=outfinal["Transported"].astype(bool)


In [265]:
outfinal.to_csv("submission.csv", index=False)

,PassengerId
0,0013_01
1,0018_01
2,0019_01
3,0021_01
4,0023_01
...,...
4272,9266_02
4273,9269_01
4274,9271_01
4275,9273_01
